In [1]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')

2023-06-13 01:35:14.353873: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 01:35:14.521008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-13 01:35:14.521082: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-13 01:35:16.348287: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [7]:
def predict(mdl, desc, service_decoder):
    probabilities = mdl.predict(x = desc)
    max_prob_idx = np.argmax(probabilities, axis=1)[0]
    prob = probabilities[0][max_prob_idx]
    labels = service_decoder[max_prob_idx]
    return labels, prob

In [3]:
with open("service_to_idx.json", "r") as enc:
    service_enc = json.load(enc)

with open("idx_to_service.json", "r") as dec:
    service_dec = json.load(dec)
    service_dec = {int(key):val for key,val in service_dec.items()}

In [5]:
new_model = tf.keras.models.load_model('saved_model/best_top_25_bert_small', compile=False)
new_model.summary()

2023-06-13 01:35:18.873369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-13 01:35:18.873511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-13 01:35:18.873611: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-13 01:35:18.873708: W tensorflow/compiler/xla/stream_executor/platform/defa

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [8]:
test = [
    """received from: efgrossm@eng.ucsd.edu Hi UCSD service desk,I am trying to access MyStudentChart, 
    but my phone broke yesterday. I cannot use DuoMobile to confirm login from my computer. 
    Can I get a bypass code while I wait for my new phone to come? -Esther
    """]
# pred = np.argmax(new_model.predict(x = test))
pred = predict(new_model, test, service_dec)
pred

1/1 [==============================] - 0s 80ms/step


('MultiFactor Authentication', 0.9658685)